# Einbettung der Einwohnerzahlen in die Ortsinformationen

In [21]:
import pandas as pd
from pathlib import Path

In [22]:
input_data_folder = "../data/"
input_data_filename = "AAB_cleaned.csv"
output_data_filename = "Kfz_pro_Ort.csv"

data_file = Path(input_data_folder) / input_data_filename
output_file = Path(input_data_folder) / output_data_filename

In [23]:
df_cars = pd.read_csv(data_file, sep= "\t")  

# Variante für Google Colab
# df_cars = pd.read_csv( "https://raw.githubusercontent.com/SebastianZug/RoboLabVortraege/refs/heads/main/30_PythonAutomobile/project/data/AAB_cleaned.csv", sep= "\t")  

## Vorverarbeitung der Kfz-Daten 

> Achtung: Dieses Notebook ignoriert aus didaktischen Gründen viele Besonderheiten im Datensatz (mehrdeutige Ortsnamen, Schreibweisen) und vereinfacht den Analyseprozess. Die abgeleiteten Aussagen sind entsprechend nicht belastbar.

In [24]:
df_counts = df_cars.groupby('Wohnort').agg(
             Anzahl=('Wohnort', 'count'),
             Kreis=('Kreis_Stadt', 'last'),
).sort_values(by='Anzahl', ascending=False).reset_index()
df_counts

,Wohnort,Anzahl,Kreis
0,Berlin,2752,Stadt Berlin
1,München,1419,Bayern
2,Hamburg,776,Hamburg
3,Dresden,631,Kreishauptmannschaft Dresden
4,Cöln,605,Rheinprovinz
...,...,...,...
9039,Klein-Lieskow b. Kottbus,1,Provinz Brandenburg
9040,Klein-Krotzenburg,1,Provinz Starkenburg
9041,Klein-Kotzenau,1,Provinz Schlesien
9042,Klein-Düben,1,Provinz Brandenburg


In [25]:
# Wie groß war die Anzahl der Fahrzeugeigentümer in Freiberg gleich?

df_counts[df_counts.Wohnort == 'Freiberg']

,Wohnort,Anzahl,Kreis
90,Freiberg,54,Kreishauptmannschaft Dresden


In [26]:
#  Lassen sie uns die kleinen Orte entfernen, die weniger als 50 Fahrzeugeigentümer haben.

df_counts = df_counts[df_counts.Anzahl >= 50]

df_counts

,Wohnort,Anzahl,Kreis
0,Berlin,2752,Stadt Berlin
1,München,1419,Bayern
2,Hamburg,776,Hamburg
3,Dresden,631,Kreishauptmannschaft Dresden
4,Cöln,605,Rheinprovinz
...,...,...,...
95,Bromberg,53,Provinz Posen
96,Wilhelmshaven,52,Provinz Hannover
97,Landshut,51,Reg.-Bezirk Niederbayern
98,Trier,51,Rheinprovinz


## Erfassen der Gemeindedaten / Einwohnerzahlen

> Vielen Dank an Herrn Ulli Schubert (info@ulischubert.de) für die Bereitstellung der Daten!

In [29]:
inhabitants_statistics_filename = "Gemeindeverzeichnis1910 mit EW.xlsx"
data_file = Path(input_data_folder) / inhabitants_statistics_filename
df_inh = pd.read_excel(data_file)


# Variante für Google Colab
# df_inh = pd.read_excel("https://github.com/SebastianZug/RoboLabVortraege/raw/refs/heads/main/30_PythonAutomobile/project/data/Gemeindeverzeichnis1910%20mit%20EW.xlsx")

In [30]:
# Wie viele Einwohner hat Freiberg 1910? Stimmt der Wert mit anderen Quellen überein?

> Soweit so gut, welche Probleme gibt es?

In [31]:
df_inh[df_inh.Gemeindename=="München"]

,Staatsform,Land,Provinzart,Provinz,Bezirksart,Bezirk,Kreisart,Kreisname,Gemeindeart,Gemeindename,Einwohner 1910,Zusatz
36306,Königreich,Preußen,Provinz,Sachsen,Regierungsbezirk,Merseburg,Kreis,Liebenwerda,Landgemeinde,München,23,NaN
52629,Königreich,Bayern,NaN,NaN,Regierungsbezirk,Oberbayern,Unmittelbare Stadt,München,Stadt,München,596467,NaN
54567,Königreich,Bayern,NaN,NaN,Regierungsbezirk,Niederbayern,Bezirksamt,Passau,Landgemeinde,München,675,NaN


In [32]:
# Die nachfolgende Lösung ist wenig belastbar und generiert falsche Ergebnisse - eine Lösung anhand der Kreisnamen und Bezirke/Länder wäre wesentlich zielführender.
# lösche die Gemeinden, deren Namen mehrfach in Gemeindename vorkommt, behalte den größeren Einwohnerwert

df_inh = df_inh.sort_values(by='Einwohner 1910', ascending=False)
df_inh = df_inh.drop_duplicates(subset='Gemeindename', keep='first')

df_inh[df_inh.Gemeindename=="München"]

,Staatsform,Land,Provinzart,Provinz,Bezirksart,Bezirk,Kreisart,Kreisname,Gemeindeart,Gemeindename,Einwohner 1910,Zusatz
52629,Königreich,Bayern,NaN,NaN,Regierungsbezirk,Oberbayern,Unmittelbare Stadt,München,Stadt,München,596467,NaN


## Merge der Daten

In [33]:
df = pd.merge(df_counts, df_inh[['Gemeindename', 'Einwohner 1910']], left_on='Wohnort', right_on='Gemeindename', how='left')

In [34]:
# lösche alle Orte, die nicht in der Einwohnerstatistik vorhanden sind

df = df.dropna(subset=['Einwohner 1910'])
df

,Wohnort,Anzahl,Kreis,Gemeindename,Einwohner 1910
0,Berlin,2752,Stadt Berlin,Berlin,2071257.0
1,München,1419,Bayern,München,596467.0
2,Hamburg,776,Hamburg,Hamburg,931035.0
3,Dresden,631,Kreishauptmannschaft Dresden,Dresden,537211.0
5,Nürnberg,460,Bayern,Nürnberg,333142.0
...,...,...,...,...,...
94,Rostock,53,Mecklenburg-Schwerin,Rostock,65383.0
95,Bromberg,53,Provinz Posen,Bromberg,57696.0
96,Wilhelmshaven,52,Provinz Hannover,Wilhelmshaven,35044.0
97,Landshut,51,Reg.-Bezirk Niederbayern,Landshut,25137.0


In [35]:
df.to_csv(output_file, sep="\t", index=False)

## Generierung von Metriken

In [36]:
df

,Wohnort,Anzahl,Kreis,Gemeindename,Einwohner 1910
0,Berlin,2752,Stadt Berlin,Berlin,2071257.0
1,München,1419,Bayern,München,596467.0
2,Hamburg,776,Hamburg,Hamburg,931035.0
3,Dresden,631,Kreishauptmannschaft Dresden,Dresden,537211.0
5,Nürnberg,460,Bayern,Nürnberg,333142.0
...,...,...,...,...,...
94,Rostock,53,Mecklenburg-Schwerin,Rostock,65383.0
95,Bromberg,53,Provinz Posen,Bromberg,57696.0
96,Wilhelmshaven,52,Provinz Hannover,Wilhelmshaven,35044.0
97,Landshut,51,Reg.-Bezirk Niederbayern,Landshut,25137.0


In [39]:
# Extraktion von Positionsinformationen

import geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="MyApp")

df['location'] = df['Wohnort'].apply(geolocator.geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)



In [40]:
df

,Wohnort,Anzahl,Kreis,Gemeindename,Einwohner 1910,location,point
0,Berlin,2752,Stadt Berlin,Berlin,2071257.0,"(Berlin, Deutschland, (52.510885, 13.3989367))","(52.510885, 13.3989367, 0.0)"
1,München,1419,Bayern,München,596467.0,"(München, Bayern, Deutschland, (48.1371079, 11...","(48.1371079, 11.5753822, 0.0)"
2,Hamburg,776,Hamburg,Hamburg,931035.0,"(Hamburg, Deutschland, (53.550341, 10.000654))","(53.550341, 10.000654, 0.0)"
3,Dresden,631,Kreishauptmannschaft Dresden,Dresden,537211.0,"(Dresden, Sachsen, Deutschland, (51.0493286, 1...","(51.0493286, 13.7381437, 0.0)"
5,Nürnberg,460,Bayern,Nürnberg,333142.0,"(Nürnberg, Bayern, Deutschland, (49.453872, 11...","(49.453872, 11.077298, 0.0)"
...,...,...,...,...,...,...,...
94,Rostock,53,Mecklenburg-Schwerin,Rostock,65383.0,"(Rostock, Mecklenburg-Vorpommern, Deutschland,...","(54.0886707, 12.1400211, 0.0)"
95,Bromberg,53,Provinz Posen,Bromberg,57696.0,"(Bydgoszcz, województwo kujawsko-pomorskie, Po...","(53.12971265, 18.02944875572119, 0.0)"
96,Wilhelmshaven,52,Provinz Hannover,Wilhelmshaven,35044.0,"(Wilhelmshaven, Niedersachsen, Deutschland, (5...","(53.5278793, 8.106301, 0.0)"
97,Landshut,51,Reg.-Bezirk Niederbayern,Landshut,25137.0,"(Landshut, Bayern, Deutschland, (48.536217, 12...","(48.536217, 12.1516551, 0.0)"


In [52]:
# plotte die Orte auf einer Karte mit Hilfe von folium

import folium

m = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

for index, row in df.iterrows():
    if row['point']:
        folium.CircleMarker(row['point'][0:2], popup=row['Wohnort']).add_to(m)

m.save('map.html')